In [18]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
import mysql.connector

#command: connection details — edit for TiDB or MySQL
config = {
    "host": "127.0.0.1",   # TiDB Cloud endpoint or localhost
    "user": "root",
    "password": "root123",
    "database": "agriculture_db",  # database name
    "port": 3307   # TiDB Cloud uses 4000
}#**config means:Unpack the dictionary and pass each key-value pair as a parameter

#command: connect to the database
conn = mysql.connector.connect(**config) #create connection b/w py and sql
cursor = conn.cursor(buffered=True) #cursor-Allows running SQL queries,stores all rows in a temporary buffer
print("✅ Connected to MySQL / XAMP DB")


✅ Connected to MySQL / XAMP DB


In [20]:
df = pd.read_csv("Cleaned_Agriculture_Data.csv")
print("📥 CSV Loaded:", df.shape)

📥 CSV Loaded: (16146, 80)


In [21]:
# 1. INSERT STATES TABLE
unique_states = df[['State Code', 'State Name']].drop_duplicates()

for _, row in unique_states.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO states (state_code, state_name)
        VALUES (%s, %s)
    """, (row['State Code'], row['State Name']))

conn.commit()
print("✅ States Inserted")

✅ States Inserted


In [22]:
# 2. INSERT DISTRICTS TABLE
unique_districts = df[['Dist Code', 'Dist Name', 'State Name']].drop_duplicates()

for _, row in unique_districts.iterrows():
    cursor.execute("SELECT state_id FROM states WHERE state_name=%s", (row['State Name'],))
    state_id = cursor.fetchone()[0]

    cursor.execute("""
        INSERT IGNORE INTO districts (dist_code, district_name, state_id)
        VALUES (%s, %s, %s)
    """, (row['Dist Code'], row['Dist Name'], state_id))

conn.commit()
print("✅ Districts Inserted")

✅ Districts Inserted


In [23]:
# 3. INSERT CROP DATA (Fact Table)
# Build the list of dataframe columns (in the exact order) that should map to DB columns.
cols = [
    'RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)', 'RICE YIELD (Kg per ha)',
    'WHEAT AREA (1000 ha)', 'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)',
    'SORGHUM AREA (1000 ha)', 'SORGHUM PRODUCTION (1000 tons)', 'SORGHUM YIELD (Kg per ha)',
    'PEARL MILLET AREA (1000 ha)', 'PEARL MILLET PRODUCTION (1000 tons)', 'PEARL MILLET YIELD (Kg per ha)',
    'MAIZE AREA (1000 ha)', 'MAIZE PRODUCTION (1000 tons)', 'MAIZE YIELD (Kg per ha)',
    'OILSEEDS AREA (1000 ha)', 'OILSEEDS PRODUCTION (1000 tons)', 'OILSEEDS YIELD (Kg per ha)',
    'SOYABEAN AREA (1000 ha)', 'SOYABEAN PRODUCTION (1000 tons)', 'SOYABEAN YIELD (Kg per ha)',
    'COTTON AREA (1000 ha)', 'COTTON PRODUCTION (1000 tons)', 'COTTON YIELD (Kg per ha)',
    'SUGARCANE AREA (1000 ha)', 'SUGARCANE PRODUCTION (1000 tons)', 'SUGARCANE YIELD (Kg per ha)',
    'FRUITS AREA (1000 ha)', 'VEGETABLES AREA (1000 ha)', 'FRUITS AND VEGETABLES AREA (1000 ha)',
    'POTATOES AREA (1000 ha)', 'ONION AREA (1000 ha)', 'FODDER AREA (1000 ha)',
    'FINGER MILLET AREA (1000 ha)', 'FINGER MILLET PRODUCTION (1000 tons)', 'FINGER MILLET YIELD (Kg per ha)',
    'BARLEY AREA (1000 ha)', 'BARLEY PRODUCTION (1000 tons)', 'BARLEY YIELD (Kg per ha)',
    'CHICKPEA AREA (1000 ha)', 'CHICKPEA PRODUCTION (1000 tons)', 'CHICKPEA YIELD (Kg per ha)',
    'PIGEONPEA AREA (1000 ha)', 'PIGEONPEA PRODUCTION (1000 tons)', 'PIGEONPEA YIELD (Kg per ha)',
    'MINOR PULSES AREA (1000 ha)', 'MINOR PULSES PRODUCTION (1000 tons)', 'MINOR PULSES YIELD (Kg per ha)',
    'GROUNDNUT AREA (1000 ha)', 'GROUNDNUT PRODUCTION (1000 tons)', 'GROUNDNUT YIELD (Kg per ha)',
    'SESAMUM AREA (1000 ha)', 'SESAMUM PRODUCTION (1000 tons)', 'SESAMUM YIELD (Kg per ha)',
    'RAPESEED AND MUSTARD AREA (1000 ha)', 'RAPESEED AND MUSTARD PRODUCTION (1000 tons)', 'RAPESEED AND MUSTARD YIELD (Kg per ha)',
    'SAFFLOWER AREA (1000 ha)', 'SAFFLOWER PRODUCTION (1000 tons)', 'SAFFLOWER YIELD (Kg per ha)',
    'CASTOR AREA (1000 ha)', 'CASTOR PRODUCTION (1000 tons)', 'CASTOR YIELD (Kg per ha)',
    'LINSEED AREA (1000 ha)', 'LINSEED PRODUCTION (1000 tons)', 'LINSEED YIELD (Kg per ha)',
    'SUNFLOWER AREA (1000 ha)', 'SUNFLOWER PRODUCTION (1000 tons)', 'SUNFLOWER YIELD (Kg per ha)'
]

# Build SQL column list including district_id and year, preserving the same order.
sql_columns = ['district_id', 'year'] + [f'`{c}`' for c in cols]

# Prepare the parameter placeholders to match the number of values dynamically.
placeholders = ', '.join(['%s'] * len(sql_columns))
columns_sql = ',\n            '.join(sql_columns)

insert_sql = f"""
    INSERT INTO crop_data (
        {columns_sql}
    ) VALUES (
        {placeholders}
    )
"""

for _, row in df.iterrows():
    cursor.execute("SELECT district_id FROM districts WHERE dist_code=%s", (row['Dist Code'],))
    district_id = cursor.fetchone()[0]

    # Build values in the same order as sql_columns
    values = [district_id, row['Year']] + [row[c] for c in cols]

    # Execute with matching number of placeholders and values
    cursor.execute(insert_sql, tuple(values))

conn.commit()
print("🌾 All crop data inserted successfully!")

cursor.close()
conn.close()


🌾 All crop data inserted successfully!


Questions to be answered:(SQL)

In [27]:
#1.Year-wise Trend of Rice Production Across States (Top 3)
query = """
SELECT 
    year,
    state_name,
    SUM(`RICE PRODUCTION (1000 tons)`) AS total_rice_production
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
JOIN states s ON d.state_id = s.state_id
GROUP BY year, state_name
ORDER BY year, total_rice_production DESC
LIMIT 3;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql')) #keywords arguments and psql postgray sql


+--------+--------------+-------------------------+
|   year | state_name   |   total_rice_production |
|--------+--------------+-------------------------|
|   1966 | West Bengal  |             4.8194e+06  |
|   1966 | Tamil Nadu   |             3.793e+06   |
|   1966 | Orissa       |             3.69179e+06 |
+--------+--------------+-------------------------+


In [28]:
#2.Top 5 Districts by Wheat Yield Increase Over the Last 5 Years
query = """
SELECT 
    d.district_name,
    (MAX(c.`WHEAT YIELD (Kg per ha)`) - MIN(c.`WHEAT YIELD (Kg per ha)`)) AS yield_increase
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
WHERE c.year >= (SELECT MAX(year) FROM crop_data) - 4
GROUP BY d.district_name
ORDER BY yield_increase DESC
LIMIT 5;
"""

cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))


+-----------------+------------------+
| district_name   |   yield_increase |
|-----------------+------------------|
| Adilabad        |          3696.11 |
| Nadia           |          3405.97 |
| Hyderabad       |          3268.42 |
| Thane           |          2833.33 |
| Kanpur          |          2750.47 |
+-----------------+------------------+


In [29]:
#3.States with the Highest Growth in Oilseed Production (5-Year Growth Rate)
query = """
SELECT 
    s.state_name,
    ((SUM(CASE WHEN c.year = (SELECT MAX(year) FROM crop_data) THEN c.`OILSEEDS PRODUCTION (1000 tons)` ELSE 0 END) -
      SUM(CASE WHEN c.year = (SELECT MAX(year) FROM crop_data) - 5 THEN c.`OILSEEDS PRODUCTION (1000 tons)` ELSE 0 END)) /
      SUM(CASE WHEN c.year = (SELECT MAX(year) FROM crop_data) - 5 THEN c.`OILSEEDS PRODUCTION (1000 tons)` ELSE 1 END)) * 100 AS growth_rate
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
JOIN states s ON d.state_id = s.state_id
GROUP BY s.state_name
ORDER BY growth_rate DESC
LIMIT 5;
"""         
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))    


+----------------+---------------+
| state_name     |   growth_rate |
|----------------+---------------|
| Andhra Pradesh |       44.6612 |
| Haryana        |       22.0023 |
| Uttarakhand    |        0      |
| Kerala         |        0      |
| Bihar          |        0      |
+----------------+---------------+


In [30]:
#4.District-wise Correlation Between Area and Production for Major Crops (Rice, Wheat, and Maize)
query = """
SELECT
    d.district_name,

    -- Rice Correlation
    (
        (
            SUM(c.`RICE AREA (1000 ha)` * c.`RICE PRODUCTION (1000 tons)`) -
            (SUM(c.`RICE AREA (1000 ha)`) * SUM(c.`RICE PRODUCTION (1000 tons)`)) / COUNT(*)
        ) /
        SQRT(
            (
                SUM(POWER(c.`RICE AREA (1000 ha)`, 2)) -
                POWER(SUM(c.`RICE AREA (1000 ha)`), 2) / COUNT(*)
            ) *
            (
                SUM(POWER(c.`RICE PRODUCTION (1000 tons)`, 2)) -
                POWER(SUM(c.`RICE PRODUCTION (1000 tons)`), 2) / COUNT(*)
            )
        )
    ) AS rice_correlation,

    -- Wheat Correlation
    (
        (
            SUM(c.`WHEAT AREA (1000 ha)` * c.`WHEAT PRODUCTION (1000 tons)`) -
            (SUM(c.`WHEAT AREA (1000 ha)`) * SUM(c.`WHEAT PRODUCTION (1000 tons)`)) / COUNT(*)
        ) /
        SQRT(
            (
                SUM(POWER(c.`WHEAT AREA (1000 ha)`, 2)) -
                POWER(SUM(c.`WHEAT AREA (1000 ha)`), 2) / COUNT(*)
            ) *
            (
                SUM(POWER(c.`WHEAT PRODUCTION (1000 tons)`, 2)) -
                POWER(SUM(c.`WHEAT PRODUCTION (1000 tons)`), 2) / COUNT(*)
            )
        )
    ) AS wheat_correlation,

    -- Maize Correlation
    (
        (
            SUM(c.`MAIZE AREA (1000 ha)` * c.`MAIZE PRODUCTION (1000 tons)`) -
            (SUM(c.`MAIZE AREA (1000 ha)`) * SUM(c.`MAIZE PRODUCTION (1000 tons)`)) / COUNT(*)
        ) /
        SQRT(
            (
                SUM(POWER(c.`MAIZE AREA (1000 ha)`, 2)) -
                POWER(SUM(c.`MAIZE AREA (1000 ha)`), 2) / COUNT(*)
            ) *
            (
                SUM(POWER(c.`MAIZE PRODUCTION (1000 tons)`, 2)) -
                POWER(SUM(c.`MAIZE PRODUCTION (1000 tons)`), 2) / COUNT(*)
            )
        )
    ) AS maize_correlation

FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
GROUP BY d.district_name
ORDER BY d.district_name;
"""
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))   

+-----------------------------------------+--------------------+---------------------+---------------------+
| district_name                           |   rice_correlation |   wheat_correlation |   maize_correlation |
|-----------------------------------------+--------------------+---------------------+---------------------|
| 24 Parganas                             |         0.451063   |         0.805426    |          0.875017   |
| Adilabad                                |         0.815215   |         0.381896    |          0.683648   |
| Agra                                    |         0.977809   |         0.8627      |          0.505339   |
| Ahmedabad                               |         0.937127   |         0.88958     |          0.873605   |
| Ahmednagar                              |         0.752331   |         0.922799    |          0.946715   |
| Ajmer                                   |         0.429769   |         0.628425    |          0.0285864  |
| Akola            

In [31]:
#5.Yearly Production Growth of Cotton in Top 5 Cotton Producing States
query = """
SELECT 
    s.state_name,
    ((SUM(CASE WHEN c.year = (SELECT MAX(year) FROM crop_data) THEN c.`COTTON PRODUCTION (1000 tons)` ELSE 0 END) -
      SUM(CASE WHEN c.year = (SELECT MAX(year) FROM crop_data) - 1 THEN c.`COTTON PRODUCTION (1000 tons)` ELSE 0 END)) /
      SUM(CASE WHEN c.year = (SELECT MAX(year) FROM crop_data) - 1 THEN c.`COTTON PRODUCTION (1000 tons)` ELSE 1 END)) * 100 AS growth_rate
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
JOIN states s ON d.state_id = s.state_id
GROUP BY s.state_name
ORDER BY growth_rate DESC
LIMIT 5;
"""
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))   


+----------------+---------------+
| state_name     |   growth_rate |
|----------------+---------------|
| Gujarat        |       96.0516 |
| Karnataka      |       79.6895 |
| Telangana      |       50.8043 |
| Andhra Pradesh |       33.4254 |
| Chhattisgarh   |       32.7381 |
+----------------+---------------+


In [35]:
#6. Districts with the Highest Groundnut Production in 2020
query = """
SELECT 
    d.district_name,
    SUM(c.`GROUNDNUT PRODUCTION (1000 tons)`) AS total_groundnut_production
FROM crop_data c    
JOIN districts d ON c.district_id = d.district_id
WHERE c.year = 2020
GROUP BY d.district_name
ORDER BY total_groundnut_production DESC
LIMIT 5;
"""

cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))   


+-----------------+------------------------------+
| district_name   | total_groundnut_production   |
|-----------------+------------------------------|
+-----------------+------------------------------+


In [ ]:
#7.Annual Average Maize Yield Across All States
query = """
SELECT 
    year,
    AVG(`MAIZE YIELD (Kg per ha)`) AS avg_maize_yield,
FROM crop_data
GROUP BY year
ORDER BY year;
"""
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))

+--------+-------------------+
|   year |   avg_maize_yield |
|--------+-------------------|
|   1966 |           734.266 |
|   1967 |           869.54  |
|   1968 |           709.262 |
|   1969 |           781.477 |
|   1970 |          1094.18  |
|   1971 |           848.643 |
|   1972 |           895.126 |
|   1973 |           886.84  |
|   1974 |           852.937 |
|   1975 |          1026.01  |
|   1976 |           966.654 |
|   1977 |           946.525 |
|   1978 |           964.877 |
|   1979 |           984.162 |
|   1980 |          1067.09  |
|   1981 |          1119.51  |
|   1982 |          1189.24  |
|   1983 |          1314.56  |
|   1984 |          1226.57  |
|   1985 |          1057.79  |
|   1986 |          1078.99  |
|   1987 |           945.444 |
|   1988 |          1145.88  |
|   1989 |          1324.73  |
|   1990 |          1235.36  |
|   1991 |          1157.02  |
|   1992 |          1450.82  |
|   1993 |          1345.81  |
|   1994 |          1211.29  |
|   1995

In [37]:
#8.Total Area Cultivated for Oilseeds in Each State
query = """
SELECT 
    s.state_name,
    SUM(c.`OILSEEDS AREA (1000 ha)`) AS total_oilseeds_area
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
JOIN states s ON d.state_id = s.state_id
GROUP BY s.state_name   
ORDER BY total_oilseeds_area DESC;
"""
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))


+------------------+-----------------------+
| state_name       |   total_oilseeds_area |
|------------------+-----------------------|
| Madhya Pradesh   |           1.89946e+08 |
| Rajasthan        |           1.42433e+08 |
| Maharashtra      |           1.34069e+08 |
| Gujarat          |           1.30725e+08 |
| Karnataka        |           9.25272e+07 |
| Andhra Pradesh   |           8.72235e+07 |
| Tamil Nadu       |           5.62256e+07 |
| Uttar Pradesh    |           4.30471e+07 |
| Orissa           |           3.9682e+07  |
| Telangana        |           3.52934e+07 |
| Kerala           |           3.43489e+07 |
| Haryana          |           2.14655e+07 |
| West Bengal      |           2.03008e+07 |
| Chhattisgarh     |           1.72671e+07 |
| Assam            |           1.41766e+07 |
| Punjab           |           8.69793e+06 |
| Bihar            |           5.53629e+06 |
| Jharkhand        |           2.82629e+06 |
| Uttarakhand      |      993530           |
| Himachal

In [38]:
#9.Districts with the Highest Rice Yield
query = """
SELECT 
    d.district_name,
    AVG(c.`RICE YIELD (Kg per ha)`) AS avg_rice_yield
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id       
GROUP BY d.district_name
ORDER BY avg_rice_yield DESC
LIMIT 5;
"""
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))


+-----------------+------------------+
| district_name   |   avg_rice_yield |
|-----------------+------------------|
| Ludhiana        |          3650.43 |
| Sangrur         |          3532.9  |
| Thirunelveli    |          3387.51 |
| Bhatinda        |          3363.16 |
| Madurai         |          3276.98 |
+-----------------+------------------+


In [1]:
#10.Compare the Production of Wheat and Rice for the Top 5 States Over 10 Years
query = """
SELECT
    s.state_name,
    c.year,
    SUM(c.`WHEAT PRODUCTION (1000 tons)`) AS total_wheat_production,
    SUM(c.`RICE PRODUCTION (1000 tons)`) AS total_rice_production   
FROM crop_data c
JOIN districts d ON c.district_id = d.district_id
JOIN states s ON d.state_id = s.state_id
GROUP BY s.state_name, c.year
ORDER BY s.state_name, c.year
LIMIT 50;
""" 
cursor.execute(query)
out = cursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[c[0] for c in cursor.description], tablefmt="psql"))   

NameError: name 'cursor' is not defined